In [3]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
from deepctr.utils import SingleFeat
from TrainingUtils import *
from deepctr.models import xDeepFM
import math

In [4]:
path='../data/train_set_final.csv'                         #请把数据的路径写在这里
data=pd.read_csv(path,low_memory=False,encoding='utf-8')

In [5]:
len(data.columns )

35

In [6]:
new_columns=['ad_id','ad_type_id','ad_account_id','item_id','item_type','size','is_all_field',
       'has_product_id', 'time', 'age', 'area', 'behavior', 'connectionType',
       'consuptionAbility', 'device', 'education', 'gender', 'status', 'work',
       'price', 'crt_dateYear', 'crt_dateMonth', 'crt_dateWeek', 'crt_dateDay',
       'crt_dateDayofweek', 'crt_dateDayofyear', 'crt_dateIs_month_end',
       'crt_dateIs_month_start', 'crt_dateIs_quarter_end',
       'crt_dateIs_quarter_start', 'crt_dateIs_year_end',
       'crt_dateIs_year_start', 'crt_dateHour', 'crt_dateElapsed','target']
data.columns=new_columns
data.columns
data['target'].head()

0    0.693147
1    0.693147
2    1.945910
3    0.693147
4    0.693147
Name: target, dtype: float64

In [7]:
p=lambda x: math.log(1+x)
data['target']=data['target'].apply(p)
data['target'].head()

0    0.526589
1    0.526589
2    1.080418
3    0.526589
4    0.526589
Name: target, dtype: float64

In [8]:
sparse_features = ['ad_id','ad_type_id','ad_account_id','item_id','item_type','is_all_field',
       'has_product_id','consuptionAbility','crt_dateYear', 'crt_dateMonth', 'crt_dateWeek', 'crt_dateDay',
       'crt_dateDayofweek', 'crt_dateDayofyear', 'crt_dateIs_month_end','crt_dateIs_month_start', 'crt_dateIs_quarter_end',
       'crt_dateIs_quarter_start', 'crt_dateIs_year_end',
       'crt_dateIs_year_start','crt_dateHour', 'crt_dateElapsed'] #在此加入稀疏特征名
dense=['price',]
multi_value_features=['size','time', 'age', 'area','device','behavior','connectionType','gender','education','area','status', 'work' ]#在此加入多值特征名
target = ['target']#在此加入标签名
hashing=False
data=sparse_feature_encoding(data,sparse_features)

In [9]:
# sequence_input,max_len_list=multi_value_feature_encoding(data,multi_value_features)

In [10]:
sparse_feat_list=sparse_feat_list_gen(data,sparse_features,hashing)
# sequence_output=sequence_feature_acquire(max_len_list,multi_value_features,hashing)

In [11]:
# 3.generate input data for model
sparse_input = [data[feat.name].values for feat in sparse_feat_list]
# dense_input = []#加入密集特征
# sequence_input = [genres_list]
model_input=sparse_input
# model_input = sparse_input + dense_input + \
#               sequence_input  # make sure the order is right

In [ ]:
# 4.Define Model,compile and train
model = xDeepFM({"sparse": sparse_feat_list,
                }, task='regression')

In [ ]:
#训练
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(model_input, data[target].values,
                    batch_size=256, epochs=1, verbose=2, validation_split=0.2, )

In [ ]:
test_path='/home/htc/code/Github/TencentAlgo/test_set_final.csv'
test_data=pd.read_csv(test_path,low_memory=False,encoding='utf-8')
pred=test_data.iloc[:,0]
# pred.columns=['样本id']
test_data=test_data.iloc[:,1:]
test_columns=new_columns[:-1]
test_data.columns=test_columns
pred.shape

In [ ]:
test_model_input=[test_data[feat.name].values for feat in sparse_feat_list]

In [ ]:
p2=lambda x: (np.exp(x) - 1)

In [ ]:
pred_ans = model.predict(test_model_input, batch_size=256)
pred_ans.shape

In [ ]:

pred_ans=np.exp(pred_ans)-1
pred_ans=pred_ans.squeeze(1)
pred=pd.DataFrame(pred)
pred_ans.shape

In [ ]:
pred['预估日曝光']=pred_ans
result_name='submission.csv'
pred.head()

In [ ]:
pred.to_csv(result_name,encoding='UTF-8')